### **Import Libraries**

In [131]:
import os 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
import pickle as pkl
from sklearn.svm import SVC

### **Loading Data**

In [45]:
# Loading t1_t2_flair_t1gd data
df_t1_t2 = pd.read_csv("../Extracted feat combined/t1_t2_flair_t1gd.csv")
df_t1_t2.drop(['Unnamed: 0'], axis=1,inplace=True)

# Loading Clinical Info
df_mgmt_clinical = pd.read_csv("../UPENN-GBM_clinical_info_v1.0.csv")
df_mgmt_clinical.rename(columns={'ID':'SubjectID'}, inplace=True)  # Renaming ID to SubjectID

# Merging the above two CSV's
df_t1_t2_mgmt = df_t1_t2.merge(df_mgmt_clinical[['Survival_from_surgery_days','SubjectID','MGMT']], on='SubjectID', how='right')
df_t1_t2_mgmt.rename(columns={'Survival_from_surgery_days':'Survival_days'}, inplace=True)
df_t1_t2_mgmt = df_t1_t2_mgmt.dropna(axis=0)
df_t1_t2_mgmt=df_t1_t2_mgmt.loc[df_t1_t2_mgmt['MGMT'].isin(['Methylated','Unmethylated'])]
df_t1_t2_mgmt=df_t1_t2_mgmt.loc[df_t1_t2_mgmt['Survival_days']!='Not Available']
print('Shape of dataset: '+str(df_t1_t2_mgmt.shape))

Shape of dataset: (209, 1731)


In [46]:
# Classifing the Survival Days into Short period and Long period
# More than 365 days are long period
# Less than or equals 365 are short period
df_t1_t2_mgmt['Survival_days']=df_t1_t2_mgmt['Survival_days'].apply(lambda x: 'long' if int(x)>365 else 'short')

# Converting long and short labels to 1 and 0 simultaneosly
df_t1_t2_mgmt['Survival_days']=df_t1_t2_mgmt['Survival_days'].apply(lambda x: 1 if x=='long' else 0)

# Methylated coded to 1 and Unmethylated coded to 0
df_t1_t2_mgmt['MGMT']=df_t1_t2_mgmt['MGMT'].apply(lambda a :1 if a=='Methylated' else 0)

# Value couts of MGMT
print(df_t1_t2_mgmt['MGMT'].value_counts())

# Value couts of Survival 
df_t1_t2_mgmt['Survival_days'].value_counts()

0    136
1     73
Name: MGMT, dtype: int64


1    109
0    100
Name: Survival_days, dtype: int64

In [47]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

## **Survival Prediction** 

In [65]:
survival_train_x, survival_test_x, survival_train_y, survival_test_y = train_test_split(df_t1_t2_mgmt.drop(['MGMT',
                                                                                                            'Survival_days',
                                                                                                            'SubjectID'],
                                                                                                           axis=1),
                                                                                        df_t1_t2_mgmt['Survival_days'], test_size=0.24,
                                                                                        random_state=100,
                                                                                        stratify=df_t1_t2_mgmt['Survival_days'])
print("Survival train size - " + str(survival_train_x.shape))
print("Survival test size - " + str(survival_test_x.shape))

Survival train size - (158, 1728)
Survival test size - (51, 1728)


In [49]:
survival_train_y.value_counts()

1    82
0    76
Name: Survival_days, dtype: int64

#### **Variance Threshold**

In [50]:
### Reducing features using Variance Threshold
threshold=0.4
low_var_col= [x for x in survival_train_x.columns if survival_train_x[x].var()<threshold]
survival_train_x.drop(low_var_col,axis=1,inplace=True)
survival_test_x.drop(low_var_col,axis=1,inplace=True)
print('Features after variance threshold: '+str(survival_train_x.shape[1]))

Features after variance threshold: 1020


#### **Correlation Coeficient**

In [51]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

correlated_idx=correlation(survival_train_x,0.85)
survival_train_x.drop(correlated_idx,axis=1,inplace=True)
survival_test_x.drop(correlated_idx,axis=1,inplace=True)
print("Shape of dataset after correlation : ",str(survival_train_x.shape))

Shape of dataset after correlation :  (158, 302)


#### **RFECV**

In [52]:
# Function for rfecv
from sklearn.feature_selection import RFE,RFECV
def rfe_feature_selection(algo,X,y,cv):
  rfe=RFECV(estimator=algo,cv=cv,min_features_to_select=20)
  rfe.fit(X,y)
  rfe_sel_index=rfe.get_support(indices=True)
  return rfe_sel_index

#### **Feature Scaling**

In [53]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
survival_x_scaled = StandardScaler().fit(survival_train_x)
survival_train_x_scaled = survival_x_scaled.transform(survival_train_x)
survival_test_x_scaled=survival_x_scaled.transform(survival_test_x)

In [54]:
import xgboost as xgb
import lightgbm as lgb
from boruta import BorutaPy
xgb_model=xgb.XGBClassifier()
survival_feat_sel_boruta=BorutaPy(xgb_model,n_estimators='auto',verbose=0,random_state=1)
survival_feat_sel_boruta.fit(survival_train_x_scaled,survival_train_y)
survival_boruta_rank=survival_feat_sel_boruta.ranking_

In [55]:
survival_boruta_rank.shape

(302,)

In [56]:
survival_col_dict_rank=dict(zip(survival_train_x.columns,survival_boruta_rank))
survival_boruta_sel_col=[col for col in survival_col_dict_rank.keys() if survival_col_dict_rank[col]<=50]
survival_boruta_train_x=survival_train_x[survival_boruta_sel_col]
survival_boruta_test_x=survival_test_x[survival_boruta_sel_col]

survival_boruta_x_scaled = StandardScaler().fit(survival_boruta_train_x)
survival_boruta_train_x_scaled = survival_boruta_x_scaled.transform(survival_boruta_train_x)
survival_boruta_test_x_scaled=survival_boruta_x_scaled.transform(survival_boruta_test_x)

In [78]:
pkl.dump(survival_boruta_train_x_scaled,open('./sel_scal_feat/survival_boruta_train_x_scaled.pkl','wb'))
pkl.dump(survival_boruta_test_x_scaled,open('./sel_scal_feat/survival_boruta_test_x_scaled.pkl','wb'))

In [105]:
pkl.dump(survival_train_y,open('./sel_scal_feat/survival_train_y.pkl','wb'))
pkl.dump(survival_test_y,open('./sel_scal_feat/survival_test_y.pkl','wb'))

In [57]:
from sklearn.svm import SVC
svc_survival=SVC(kernel='rbf',C=0.1)
svc_survival.fit(survival_boruta_train_x_scaled,survival_train_y)
print('Score of train: ',str(svc_survival.score(survival_boruta_train_x_scaled,survival_train_y)))
print('Score of test: ',str(svc_survival.score(survival_boruta_test_x_scaled,survival_test_y)))

Score of train:  0.5189873417721519
Score of test:  0.5294117647058824


In [58]:
y_pred_survival = svc_survival.predict(survival_boruta_test_x_scaled)
generate_model_report(survival_test_y, y_pred_survival)

Accuracy =  0.5294117647058824
Precision =  0.5294117647058824
Recall =  1.0
F1 Score =  0.6923076923076924


In [59]:
# svc.fit(survival_boruta_train_x_scaled)
index_survival = svc_survival.decision_function(survival_boruta_train_x_scaled)
len(index_survival)

158

In [109]:
index_survival_test = svc_survival.decision_function(survival_boruta_test_x_scaled)
len(index_survival_test)

51

In [110]:
pkl.dump(index_survival, open('./svm_indices/index_survival.pkl', 'wb'))
pkl.dump(index_survival_test, open('./svm_indices/index_survival_test.pkl', 'wb'))

## **MGMT**

In [66]:
mgmt_train_x, mgmt_test_x, mgmt_train_y, mgmt_test_y = train_test_split(df_t1_t2_mgmt.drop(['Survival_days','SubjectID','MGMT'],axis=1), 
                                                    df_t1_t2_mgmt['MGMT'], test_size=0.24, 
                                                    random_state=100, 
                                                    stratify=df_t1_t2_mgmt['MGMT'])
print("train size - " + str(mgmt_train_x.shape))
print("test size - " + str(mgmt_test_x.shape))

train size - (158, 1728)
test size - (51, 1728)


In [67]:
mgmt_train_y.value_counts()

0    103
1     55
Name: MGMT, dtype: int64

#### **Variance Threshold**

In [68]:
### Reducing features using Variance Threshold
threshold=0.4
low_var_col= [x for x in mgmt_train_x.columns if mgmt_train_x[x].var()<threshold]
mgmt_train_x.drop(low_var_col,axis=1,inplace=True)
mgmt_test_x.drop(low_var_col,axis=1,inplace=True)

In [69]:
mgmt_correlated_idx=correlation(mgmt_train_x,0.85)
mgmt_train_x.drop(mgmt_correlated_idx,axis=1,inplace=True)
mgmt_test_x.drop(mgmt_correlated_idx,axis=1,inplace=True)
print("Shape of dataset: ",str(mgmt_train_x.shape))

Shape of dataset:  (158, 298)


In [70]:
mgmt_x_scaled = StandardScaler().fit(mgmt_train_x)
mgmt_train_x_scaled = mgmt_x_scaled.transform(mgmt_train_x)
mgmt_test_x_scaled=mgmt_x_scaled.transform(mgmt_test_x)

In [71]:
import xgboost as xgb
import lightgbm as lgb
from boruta import BorutaPy
xgb_model=xgb.XGBClassifier()
mgmt_feat_sel_boruta=BorutaPy(xgb_model,n_estimators='auto',verbose=0,random_state=1)
mgmt_feat_sel_boruta.fit(mgmt_train_x_scaled,survival_train_y)
mgmt_boruta_rank=mgmt_feat_sel_boruta.ranking_
mgmt_boruta_rank.shape

(298,)

In [73]:
mgmt_col_dict_rank=dict(zip(mgmt_train_x.columns,mgmt_boruta_rank))
mgmt_boruta_sel_col=[col for col in mgmt_col_dict_rank.keys() if mgmt_col_dict_rank[col]<=50]
mgmt_boruta_train_x=mgmt_train_x[mgmt_boruta_sel_col]
mgmt_boruta_test_x=mgmt_test_x[mgmt_boruta_sel_col]

mgmt_boruta_x_scaled = StandardScaler().fit(mgmt_boruta_train_x)
mgmt_boruta_train_x_scaled = mgmt_boruta_x_scaled.transform(mgmt_boruta_train_x)
mgmt_boruta_test_x_scaled=mgmt_boruta_x_scaled.transform(mgmt_boruta_test_x)

In [79]:
pkl.dump(mgmt_boruta_train_x_scaled,open('./sel_scal_feat/mgmt_boruta_train_x_scaled.pkl','wb'))
pkl.dump(mgmt_boruta_test_x_scaled,open('./sel_scal_feat/mgmt_boruta_test_x_scaled.pkl','wb'))

In [111]:
pkl.dump(mgmt_train_y,open('./sel_scal_feat/mgmt_train_y.pkl','wb'))
pkl.dump(mgmt_test_y,open('./sel_scal_feat/mgmt_test_y.pkl','wb'))

In [74]:
from sklearn.svm import SVC
from sklearn.metrics import recall_score
svc_mgmt=SVC(kernel='linear',class_weight='balanced',C=0.3)
svc_mgmt.fit(mgmt_boruta_train_x_scaled,mgmt_train_y)
print('Score of train: ',str(svc_mgmt.score(mgmt_boruta_train_x_scaled,mgmt_train_y)))
print('Score of test: ',str(svc_mgmt.score(mgmt_boruta_test_x_scaled,mgmt_test_y)))
y_pred=svc_mgmt.predict(mgmt_boruta_test_x_scaled)
print(y_pred)
print(recall_score(mgmt_test_y,y_pred))

Score of train:  0.6772151898734177
Score of test:  0.37254901960784315
[1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0
 0 0 1 1 0 1 1 0 1 1 1 1 1 1]
0.3888888888888889


In [75]:
generate_model_report(mgmt_test_y,y_pred)

Accuracy =  0.37254901960784315
Precision =  0.25
Recall =  0.3888888888888889
F1 Score =  0.30434782608695654


In [76]:
# svc.fit(survival_boruta_train_x_scaled)
index_mgmt = svc_mgmt.decision_function(mgmt_boruta_train_x_scaled)
len(index_mgmt)

158

In [112]:
index_mgmt_test = svc_mgmt.decision_function(mgmt_boruta_test_x_scaled)
len(index_mgmt_test)

51

In [113]:
pkl.dump(index_mgmt, open('./svm_indices/index_mgmt.pkl', 'wb'))
pkl.dump(index_mgmt_test, open('./svm_indices/index_mgmt_test.pkl', 'wb'))

#### GridsearchCV for weights

In [29]:
from sklearn.model_selection import GridSearchCV
weights = np.linspace(0.05, 0.95, 20)
param_grid = dict(class_weight=[{0:x, 1:1.0-x} for x in weights])
gsc = GridSearchCV(
    estimator=SVC(kernel='linear', C=0.3),
    param_grid=param_grid,
    scoring='f1',
    cv=5
)
grid_result = gsc.fit(mgmt_boruta_train_x_scaled, mgmt_train_y)
best_params = grid_result.best_params_
print(best_params)

{'class_weight': {0: 0.09736842105263158, 1: 0.9026315789473685}}


In [30]:
mgmt_svc = SVC(kernel='linear',C=0.3, class_weight=best_params['class_weight'])
mgmt_model = svc.fit(mgmt_boruta_train_x_scaled, mgmt_train_y)

In [31]:
y_pred_mgmt = mgmt_model.predict(mgmt_boruta_test_x_scaled)
generate_model_report(mgmt_test_y, y_pred_mgmt)

Accuracy =  0.37254901960784315
Precision =  0.25
Recall =  0.3888888888888889
F1 Score =  0.30434782608695654


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

pipe = make_pipeline(
    SMOTE(), 
    SVC(kernel='linear',C=0.3, class_weight=best_params['class_weight'])
)
weights = np.linspace(0.005, 0.75, 100)


gsc_smote = GridSearchCV(
    estimator=pipe,
    param_grid={
        'smote__sampling_strategy': weights
    },
    scoring='f1',
    cv=5
)
grid_result = gsc_smote.fit(mgmt_boruta_train_x_scaled, mgmt_train_y)
best_params = grid_result.best_params_
print(best_params)


## crossing the indices

### mgmt index in survival prediction

In [148]:
survival_boruta_train_x_scaled = pkl.load(open('./sel_scal_feat/survival_boruta_train_x_scaled.pkl', 'rb'))
survival_boruta_test_x_scaled = pkl.load(open('./sel_scal_feat/survival_boruta_test_x_scaled.pkl', 'rb'))

index_mgmt_train = pkl.load(open('./svm_indices/index_mgmt.pkl', 'rb'))
index_mgmt_test = pkl.load(open('./svm_indices/index_mgmt_test.pkl', 'rb'))

In [149]:
survival_train_y = pkl.load(open('./sel_scal_feat/survival_train_y.pkl', 'rb'))
survival_test_y = pkl.load(open('./sel_scal_feat/survival_test_y.pkl', 'rb'))

In [150]:
survival_train_crossed = np.column_stack((survival_boruta_train_x_scaled, index_mgmt))
survival_test_crossed = np.column_stack((survival_boruta_test_x_scaled, index_mgmt_test))
print(survival_train_crossed.shape)
print(survival_test_crossed.shape)

(158, 50)
(51, 50)


In [154]:
svc_survival_crossed = SVC(kernel='linear', C=0.3)
svc_survival_crossed.fit(survival_train_crossed, survival_train_y)
print('Score of train: ',str(svc_survival_crossed.score(survival_train_crossed, survival_train_y)))
print("Test results")
y_pred_survival_crossed=svc_survival_crossed.predict(survival_test_crossed)
generate_model_report(survival_test_y,y_pred_survival_crossed)

Score of train:  0.8227848101265823
Test results
Accuracy =  0.5294117647058824
Precision =  0.5789473684210527
Recall =  0.4074074074074074
F1 Score =  0.47826086956521735


### survival index cross with mgmt prediction

In [137]:
mgmt_boruta_train_x_scaled = pkl.load(open('./sel_scal_feat/mgmt_boruta_train_x_scaled.pkl', 'rb'))
mgmt_boruta_test_x_scaled = pkl.load(open('./sel_scal_feat/mgmt_boruta_test_x_scaled.pkl', 'rb'))

index_survival_train = pkl.load(open('./svm_indices/index_survival.pkl', 'rb'))
index_survival_test = pkl.load(open('./svm_indices/index_survival_test.pkl', 'rb'))

mgmt_train_y = pkl.load(open('./sel_scal_feat/mgmt_train_y.pkl', 'rb'))
mgmt_test_y = pkl.load(open('./sel_scal_feat/mgmt_test_y.pkl', 'rb'))

In [138]:
mgmt_train_crossed = np.column_stack((mgmt_boruta_train_x_scaled, index_survival_train))
mgmt_test_crossed = np.column_stack((mgmt_boruta_test_x_scaled, index_survival_test))
print(mgmt_train_crossed.shape)
print(mgmt_test_crossed.shape)

(158, 48)
(51, 48)


In [146]:
svc_mgmt_crossed = SVC(kernel='linear', C=0.3)
svc_mgmt_crossed.fit(mgmt_train_crossed, mgmt_train_y)
print('Score of train: ',str(svc_mgmt_crossed.score(mgmt_train_crossed, mgmt_train_y)))
print("Test results")
y_pred_mgmt_crossed=svc_mgmt_crossed.predict(mgmt_test_crossed)
generate_model_report(mgmt_test_y,y_pred_mgmt_crossed)


Score of train:  0.7531645569620253
Test results
Accuracy =  0.6078431372549019
Precision =  0.375
Recall =  0.16666666666666666
F1 Score =  0.23076923076923078
